In [1]:
#Step 1: Install dependencies
!pip install vina
!apt-get install -y autodock-vina

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 11.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  autodock autogrid
The following NEW packages will be installed:
  autodock-vina
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 282 kB of archives.
After this operation, 714 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 autodock-vina amd64 1.2.3-2 [282 kB]
Fetched 282 kB in 1s (296 kB/s)
Selecting previously unselected package autodock-vina.
(Reading database ... 126441 files and directories currently installed.)
Preparing to unpack .../autodock-vina_1.2.3-2_amd64.deb ...
Unpacking autodock-vina (1.2.3-2) ...
Setting up autodock-vina (1.2.3-2) ...
Processing triggers for man-db (2.10.2-1) ...


In [2]:
#Step 2: Upload your protein and ligand files
from google.colab import files
uploaded = files.upload() # Upload protein.pdbqt and ligand.pdbqt
# If you have multiple ligands as a zip file
!unzip ligands.zip -d ligands/

Saving NS3_model.pdbqt to NS3_model.pdbqt
unzip:  cannot find or open ligands.zip, ligands.zip.zip or ligands.zip.ZIP.


In [3]:
#Step 2: Upload your protein and ligand files
from google.colab import files
uploaded = files.upload() # Upload protein.pdbqt and ligand.pdbqt
# If you have multiple ligands as a zip file
# !unzip ligands.zip -d ligands/

Saving Simeprevir.pdbqt to Simeprevir.pdbqt


In [4]:
#Step 3: Docking script for single ligand
from vina import Vina
import os

# Define input filenames
receptor_file = '/content/NS3_model.pdbqt'
ligand_file = '/content/Simeprevir.pdbqt'
corrected_receptor_file = '/content/NS3_model_corrected.pdbqt'

# Attempt to fix the PDBQT receptor file by removing lines that don't contain atom information
try:
    with open(receptor_file, 'r') as infile, open(corrected_receptor_file, 'w') as outfile:
        for line in infile:
            # Keep lines that start with 'ATOM', 'HETATM', or 'END'
            if line.startswith('ATOM') or line.startswith('HETATM') or line.startswith('END'):
                outfile.write(line)

    print(f"Corrected PDBQT receptor file created as {corrected_receptor_file}")

except FileNotFoundError:
    print(f"Error: Receptor file not found at {receptor_file}")
    # Optionally exit or handle the error appropriately
    exit() # Or you could skip docking and continue with other tasks

v = Vina(sf_name='vina')

# Use the corrected receptor file
v.set_receptor(corrected_receptor_file)
v.set_ligand_from_file(ligand_file)

# Define docking box (adjust values for your protein binding site)
# Make sure these values are appropriate for your specific protein
center = [10,20,30]
box_size = [20,20,20]

# Compute affinity maps for the receptor
v.compute_vina_maps(center=center, box_size=box_size)

# Perform docking
v.dock(exhaustiveness=8, n_poses=10)

# Write the docking poses to a file
output_docked_file = 'docked_ligand.pdbqt'
v.write_poses(output_docked_file, n_poses=5, overwrite=True)

print(f"Docking completed. Docked poses saved to {output_docked_file}")

# Print the docking scores
print("Docking Scores:", v.poses(n_poses=5))

Corrected PDBQT receptor file created as /content/NS3_model_corrected.pdbqt
Docking completed. Docked poses saved to docked_ligand.pdbqt
Docking Scores: MODEL 1
REMARK VINA RESULT:   -10.803      0.000      0.000
REMARK INTER + INTRA:          62.740
REMARK INTER:                 -15.223
REMARK INTRA:                  77.963
REMARK UNBOUND:                77.963
REMARK  Name = 24873435
REMARK  13 active torsions:
REMARK  status: ('A' for Active; 'I' for Inactive)
REMARK    1  A    between atoms: S_1  and  N_11
REMARK    2  A    between atoms: S_1  and  C_21
REMARK    3  A    between atoms: O_4  and  C_28
REMARK    4  A    between atoms: O_4  and  C_35
REMARK    5  A    between atoms: O_9  and  C_44
REMARK    6  A    between atoms: O_9  and  C_50
REMARK    7  A    between atoms: N_12  and  C_36
REMARK    8  A    between atoms: C_15  and  C_18
REMARK    9  A    between atoms: C_40  and  C_45
REMARK   10  A    between atoms: C_42  and  C_46
REMARK   11  A    between atoms: C_47  and  C_49